In [ ]:
import os
import mne
from utils.config import BASE_DIR, EAR_ELECTRODES
from utils import compare_raw_and_file_annotations

mne.set_log_level("error")
# Set MNE configuration for light theme
mne.set_config('MNE_BROWSER_THEME', 'light')  # Set browser theme to light
mne.viz.set_browser_backend('qt')
# Alternative approach: set Qt style
os.environ['QT_STYLE_OVERRIDE'] = 'Fusion'  # Use Fusion style which is lighter

In [2]:
sub = "DG1"
sub_dir = f"{BASE_DIR}/control_clean/{sub}"
filename = f"{sub}_196-head-ch_resample250_filtered_scored_bad-epochs"
bad_channels_path = f"{sub_dir}/{sub}_bad_channels.txt"

In [ ]:
raw = mne.io.read_raw(f"{sub_dir}/{filename}.fif")
channels = [ch for ch in raw.ch_names if ch not in EAR_ELECTRODES]
raw.pick(channels)

In [4]:
def sort_electrode_names(electrode_list):
    return sorted(electrode_list, key=lambda x: int(x[1:]) if x.startswith('E') and x[1:].isdigit() else float('inf'))

In [ ]:
print("annotations.txt")
print("-" * 20)
compare_raw_and_file_annotations(raw, f"{sub_dir}/{sub}_annotations.txt")
print("\ncleaned_annotations.txt")
print("-" * 20)
_ = compare_raw_and_file_annotations(raw, f"{sub_dir}/{sub}_cleaned_annotations.txt")

annotations.txt
--------------------
Raw file annotations: 239
Text file annotations: 239
✓ All annotations match!

cleaned_annotations.txt
--------------------
Raw file annotations: 239
Text file annotations: 228
⚠️ Different number of annotations!


In [14]:
# Load cleaned annotations if they exist
cleaned_annotations_path = f"{sub_dir}/{sub}_cleaned_annotations.txt"
if os.path.exists(cleaned_annotations_path):
    cleaned_annotations = mne.read_annotations(cleaned_annotations_path)
    raw.set_annotations(cleaned_annotations)
    print(f"Loaded {len(cleaned_annotations)} annotations from: {cleaned_annotations_path}")
else:
    print(f"Cleaned annotations file not found: {cleaned_annotations_path}")

Loaded 228 annotations from: I:/Shaked/ISO_data/control_clean/DG1/DG1_cleaned_annotations.txt


In [9]:
# Load bad channels from file and set them as bad
if os.path.exists(bad_channels_path):
    with open(bad_channels_path, 'r') as f:
        loaded_bad_channels = [line.strip() for line in f if line.strip()]
    
    raw.info['bads'] = loaded_bad_channels
    print(f"Loaded and set {len(loaded_bad_channels)} bad channels from file:")
    print(f"Bad channels: {sort_electrode_names(loaded_bad_channels)}")
else:
    print(f"Bad channels file not found: {bad_channels_path}")

Loaded and set 11 bad channels from file:
Bad channels: ['E19', 'E82', 'E91', 'E102', 'E111', 'E182', 'E183', 'E193', 'E194', 'E216', 'E229']


### Look at the channels and mark bad ones!

In [10]:
raw.plot(duration=50)

In [15]:
bad_channels = sort_electrode_names([str(x) for x in raw.info['bads']])
bad_percentage = (len(bad_channels) / len(raw.ch_names)) * 100
print(f"Bad channels marked: {bad_channels}")
print(f"Count: {len(bad_channels)} ({bad_percentage:.2f}%)")

Bad channels marked: ['E19', 'E82', 'E91', 'E102', 'E111', 'E112', 'E120', 'E182', 'E183', 'E193', 'E194', 'E216', 'E229']
Count: 13 (6.63%)


In [16]:
with open(bad_channels_path, 'w') as f:
    for ch in bad_channels:
        f.write(f"{ch}\n")

In [ ]:
_ = compare_raw_and_file_annotations(raw, f"{sub_dir}/{sub}_cleaned_annotations.txt")
# _ = compare_raw_and_file_annotations(raw, f"{sub_dir}/{sub}_annotations.txt")

Raw file annotations: 228
Text file annotations: 228
✓ All annotations match!


In [14]:
# Save current annotations to cleaned_annotations.txt
cleaned_annotations_path = f"{sub_dir}/{sub}_cleaned_annotations.txt"
raw.annotations.save(cleaned_annotations_path, overwrite=True)
print(f"Saved {len(raw.annotations)} annotations to: {cleaned_annotations_path}")

Saved 228 annotations to: I:/Shaked/ISO_data/control_clean/DG1/DG1_cleaned_annotations.txt
